In [ ]:
import os
from dotenv import load_dotenv
import json
import random
import matplotlib.pyplot as plt
import numpy as np
import asyncio

from benchmarklib import BenchmarkDatabase
from clique import CliqueProblem, CliqueTrial
from benchmarklib.compilers import CompileType, XAGCompiler, QCFCompiler

from benchmarklib.analysis import analyze_success_rates
from benchmarklib.compilers import TruthTableCompiler

from qiskit_ibm_runtime import QiskitRuntimeService

In [ ]:
service = QiskitRuntimeService()
benchmark_db = BenchmarkDatabase("clique.db", CliqueProblem, CliqueTrial)

In [ ]:
# run this cell to load the job results for all trials that are waiting pending job results
await benchmark_db.update_all_pending_results(service, batch_size=1)

In [ ]:

    
analyze_success_rates(benchmark_db, (QCFCompiler(), XAGCompiler()), "plots")
